In [1]:
import numpy as np

# Format of the output dictionary
wnt_residue_mapping = {
    'wnt1': {
        'original_residues': {
            'residue_ids': [],
            'residue_labels': []
        },
        'wnt8a_alignment' : {'residue_ids': []}
    },
    'wnt3a': {
        'original_residues': {
            'residue_ids': [],
            'residue_labels': []
        },
        'wnt8a_alignment' : {'residue_ids': []}
    },
    'wnt5a': {
        'original_residues': {
            'residue_ids': [],
            'residue_labels': []
        },
        'wnt8a_alignment' : {'residue_ids': []}
    },
    'wnt8a': {
        'original_residues': {
            'residue_ids': [],
            'residue_labels': []
        },
        'wnt8a_alignment' : {'residue_ids': []}
    }
}

In [2]:
systems = ["1", "3a", "5a", "8a"]
resnames_from_pdb = np.empty(len(systems), dtype=object)
system_idx_from_mapping = np.empty(len(systems), dtype=object)
mapping_idx_from_mapping = np.empty(len(systems), dtype=object)

for i in range(len(systems)):
    
    # Start with a blank list
    resnames_from_pdb[i] = []
    system_idx_from_mapping[i] = []
    mapping_idx_from_mapping[i] = []
    
    # Load in the mapping files generated from 02*ipynb
    data = np.loadtxt(f"output/{systems[i]}_to_8a_map.csv", delimiter=",", skiprows=1, dtype=str)
    
    # First col = 1a/3a/5a/8a indeces
    curr_idx = data[:,0].tolist()
    
    # Second sol = 8a mapping indeces
    map_idx = data[:,1].tolist()
    
    # Load in the pdb files and read in the residue name
    col = np.loadtxt(f"input/Wnt{systems[i]}_align.pdb", comments=["CRYST1", "TER", "END"], dtype=str)
    if systems[i] in ["3a", "8a"]:
        resnames = col[:,3] + col[:,5]
    else:
        resnames = col[:,3] + col[:,4]
    
    # Read in molecule id (WNT or WNTLESS) and generate list of WNT resnames
    seg = col[:,-2]
    for res in range(len(resnames)):
        if resnames[res] not in resnames_from_pdb[i] and seg[res] == "PROA":
            resnames_from_pdb[i].append(str(resnames[res]))
            
    # Output
    resnames_from_pdb[i] = resnames_from_pdb[i]
    system_idx_from_mapping[i] = curr_idx
    mapping_idx_from_mapping[i] = map_idx
    

/var/folders/p_/mmkyzkc1655_rf8nypyzqx7r0000gn/T/ipykernel_12734/3397276745.py:23: UserWarning: Input line 5245 contained no data and will not be counted towards `max_rows=50000`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an example on how to do this.  If you wish to ignore this warning, use `warnings.filterwarnings`.  This warning is expected to be removed in the future and is given only once per `loadtxt` call.
  col = np.loadtxt(f"input/Wnt{systems[i]}_align.pdb", comments=["CRYST1", "TER", "END"], dtype=str)
/var/folders/p_/mmkyzkc1655_rf8nypyzqx7r0000gn/T/ipykernel_12734/3397276745.py:23: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=50000`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by

In [4]:
# Take arrays and store in the dictionary
# Print out sizes to make sure number of elements in each dictionary item are the same
for i in range(len(systems)):
    dict_item = f"wnt{systems[i]}"
    print(f"Number of Residues from PDB: {len(system_idx_from_mapping[i])}\nNumber of Residues from Map: {len(resnames_from_pdb[i])}\nNumber of aligned from map:{len(mapping_idx_from_mapping[i])}\n")
    wnt_residue_mapping[dict_item]['wnt8a_alignment']['residue_ids'] = mapping_idx_from_mapping[i]
    wnt_residue_mapping[dict_item]['original_residues']['residue_ids'] = system_idx_from_mapping[i]
    wnt_residue_mapping[dict_item]['original_residues']['residue_labels'] = resnames_from_pdb[i]

Number of Residues from PDB: 338
Number of Residues from Map: 338
Number of aligned from map:338

Number of Residues from PDB: 334
Number of Residues from Map: 334
Number of aligned from map:334

Number of Residues from PDB: 337
Number of Residues from Map: 337
Number of aligned from map:337

Number of Residues from PDB: 316
Number of Residues from Map: 316
Number of aligned from map:316

